In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import RandomizedSearchCV
#from xgboost import XGBRegressor
import warnings

In [3]:
df = pd.read_csv('data/raw.csv')

In [4]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
x = df.drop(columns=['math score'], axis = 1)

In [6]:
x.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [7]:
y = df['math score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [9]:
# Transforming the numerical and categorical features

num_features = x.select_dtypes(exclude='object').columns
cat_features = x.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

ohe = OneHotEncoder()
ss = StandardScaler()

preprocessor = ColumnTransformer(
  [
    ("OneHotEncoder",ohe,cat_features),
    ("Standardscaler",ss,num_features)
  ]
)

x = preprocessor.fit_transform(x)

In [12]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [13]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((800, 19), (200, 19))

In [14]:
# evaluate function

def evaluate_model(true,predicted):
  mae = mean_absolute_error(true,predicted)
  mse = mean_squared_error(true,predicted)
  rmse = np.sqrt(mse)
  r2 = r2_score(true,predicted)
  
  return mae,mse,rmse,r2

In [16]:
models = {
  "LinearRegression":LinearRegression(),
  "Lasso":Lasso(),
  "Ridge":Ridge(),
  "KNN":KNeighborsRegressor(),
  "Decision Tree":DecisionTreeRegressor(),
  "Random Forest":RandomForestRegressor(),
  "AdaBoost":AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train,y_train)
  
  train_pred = model.predict(x_train)
  test_pred = model.predict(x_test)
  
  train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train,train_pred)
  test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test,test_pred)
  
  
  print('\n')
  print(list(models.keys())[i])
  model_list.append(list(models.keys())[i])
  
  print("Model performance for training set:")
  print('Mean squared error: {:.4f}'.format(train_mse))
  print('Mean absolute error: {:.4f}'.format(train_mae))
  print('Root Mean squared error: {:.4f}'.format(train_rmse))
  print('R2 score: {:.4f}'.format(train_r2))
  
  print('------------------------------------------')
  
  print("Model performance for testing set:")
  print('Mean squared error: {:.4f}'.format(test_mse))
  print('Mean absolute error: {:.4f}'.format(test_mae))
  print('Root Mean squared error: {:.4f}'.format(test_rmse))
  print('R2 score: {:.4f}'.format(test_r2))
  
  
  



LinearRegression
Model performance for training set:
Mean squared error: 28.3485
Mean absolute error: 4.2671
Root Mean squared error: 5.3243
R2 score: 0.8743
------------------------------------------
Model performance for testing set:
Mean squared error: 29.1167
Mean absolute error: 4.2158
Root Mean squared error: 5.3960
R2 score: 0.8803


Lasso
Model performance for training set:
Mean squared error: 43.4784
Mean absolute error: 5.2063
Root Mean squared error: 6.5938
R2 score: 0.8071
------------------------------------------
Model performance for testing set:
Mean squared error: 42.5064
Mean absolute error: 5.1579
Root Mean squared error: 6.5197
R2 score: 0.8253


Ridge
Model performance for training set:
Mean squared error: 28.3378
Mean absolute error: 4.2650
Root Mean squared error: 5.3233
R2 score: 0.8743
------------------------------------------
Model performance for testing set:
Mean squared error: 29.0563
Mean absolute error: 4.2111
Root Mean squared error: 5.3904
R2 score: 